In [ ]:
import warnings
from trainer import *
from models import *
from datamodule import Dataset
warnings.filterwarnings("ignore")
from utils import *
from car_env import *

In [ ]:
dataset = Dataset(path = './data')
dataset.summarize()

In [ ]:
car_env = CarEnv()
model = Model(5)
trainer = Trainer(max_epochs = 10)
trainer.fit(model, dataset)
evaluate(model, dataset)

In [ ]:
car_env.play(model)

: 